In [27]:
import pandas as pd
import glob
from snownlp import SnowNLP
import re
from transformers import AutoTokenizer, AutoModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from transformers import TFBertForSequenceClassification, BertTokenizer
import tensorflow as tf
import torch
import jieba
import math
from collections import Counter
from collections import defaultdict

In [28]:
# 直接跑jms的csv
# readpath = r'data/processing/'
# readpath = r'./rationality/'
# dataframe = pd.read_csv(readpath+'precessed.csv')
# dataframe = dataframe.head()
# dataframe = pd.read_csv(readpath+'processed(ver3.0).csv')
dataframe = pd.read_csv('precessed.csv')

## 函数定义

In [ ]:
# 返回的是negative的概率；positive的概率就是1-negative
# def get_sentiment(text):
#     model_name_or_path = "./local_model"
#     # 加载分词器
#     tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

#     # 加载模型
#     model = AutoModel.from_pretrained(model_name_or_path)

#     # tokenizer = BertTokenizer.from_pretrained(model_name_or_path)
#     # model = TFBertForSequenceClassification.from_pretrained(model_name_or_path, from_pt=True)
#     # tokenizer = BertTokenizer.from_pretrained('IDEA-CCNL/Erlangshen-Roberta-110M-Sentiment')
#     # model = TFBertForSequenceClassification.from_pretrained('IDEA-CCNL/Erlangshen-Roberta-110M-Sentiment', from_pt=True)

#     inputs = tokenizer(text, return_tensors='tf')
#     output = model(**inputs)

#     probs = tf.nn.softmax(output.logits, axis=-1)
#     probs_np = probs.numpy()  # Convert to numpy array

#     prob_class1 = probs_np[0, 0]  # Probability of the first class
#     prob_class2 = probs_np[0, 1]  # Probability of the second class

#     return prob_class1

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch


model_name_or_path = "./local_model"
# 加载分词器
# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

# # 加载模型
# model = AutoModel.from_pretrained(model_name_or_path)
# 加载分词器和模型
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path)

def get_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    negative_prob = probs[0][0].item()  # 假设第一个标签是负面情感
    return negative_prob

# 情感获取
def sentiment(list):
    negative_prob = []
    for item in list:
        if isinstance(item, str):
            negative = get_sentiment(item)
            negative_prob.append(negative)
        else:
            negative_prob.append(None)
    return negative_prob

In [30]:
# 标题文本处理
def process_title(data):
    titles = []
    for title in data:
        if isinstance(title, str):
            if title[:4] == '慈善募捐':
                titles.append(title[7:-7])
            else:titles.append(title)
        else:
            titles.append(title)
    return titles

In [31]:
#项目介绍文本处理
def process_detail(data):

    patterns = [r'（[^（）]*?图[^（）]*?）', r'【[^【】]*?图[^【】]*?】', r'（[^（）]*?照片[^（）]*?）', r'【[^【】]*?照片[^【】]*?】']
    photo = [] # 照片数量
    detail_ = [] # 剔除这类信息后的文本
    details = [] # 最终返回的文本

    for detail in data:
        if isinstance(detail, str):
            match = []
            for pattern in patterns:
                match = match + re.findall(pattern, detail, flags=0)
                detail = re.sub(pattern, '', detail)
            photo.append(len(match))
            detail_.append(detail)
        else:
            detail_.append('')
            photo.append(0)

    for detail in detail_:
        if detail != '':
            text = ''.join(detail.split(r"', '"))
            details.append(text)
        else:
            details.append(detail)


    return photo, details


# jieba分词，返回分出来的词，同时剔除非中文非数字字符；处理对象为文本
def clean(text):

    pattern = r'[^\u4e00-\u9fa5\d]' #用于剔除非中文非数字字符
    clean_words = []

    if isinstance(text, str):
        words = jieba.lcut(text)

        for word in words:
            clean_word = re.sub(r'[^\u4e00-\u9fa5\d]', '', word)
            if clean_word != '':
                clean_words.append(clean_word)

    return clean_words


# 信息量，计算clean后分词的数量
def get_info(lst):

    info = []

    for item in lst:
        clean_words = clean(item)
        info.append(len(clean_words))

    return info


# 有强烈情感的符号
def get_mark(data):

    Exclamation = []
    QMark = []

    for text in data:
        words = jieba.lcut(text)
        Exclamation.append(words.count('？')+words.count('?'))
        QMark.append(words.count('！'))

    return Exclamation, QMark

In [32]:
# 计算entropy
def document_frequency(list_of_text):

    # Initialize the document frequency dictionary
    document_frequency = defaultdict(int)

    # Iterate over each text and update the document frequency dictionary
    for text in list_of_text:
        if isinstance(text, str):
            words = jieba.lcut(text)
            for word in words:
                document_frequency[word] += 1
    return document_frequency


def calculate_entropy(text, list_of_text):
    # 需要先有document_frequency，可以用上一个函数获取。
    # 返回的是输入内容的entropy
    if isinstance(text, str):
        words = jieba.lcut(text)
        word_counts = Counter(words)
        total_words = len(words)
        entropy = 0.0

        for word, count in word_counts.items():
            probability = count / total_words
            inverse_document_frequency = math.log((len(list_of_text) + 1) / (document_frequency[word] + 1))
            entropy += inverse_document_frequency * probability * math.log(probability, 2)

        entropy = -entropy
    else:
        entropy = None
    return entropy



## 数据处理

### 一起跑（后面的就不用跑了）

In [ ]:
# Title
titles = process_title(dataframe['项目名称'].tolist())
TInfo = get_info(titles)
TEmo = sentiment(titles)
dataframe['TInfo'] = TInfo
dataframe['TEmo'] = TEmo

# Brief
briefs = dataframe['项目简介'].tolist()
BInfo = get_info(briefs)
BEmo = sentiment(briefs)
dataframe['BInfo'] = BInfo
dataframe['BEmo'] = BEmo

# Detail
photo, details = process_detail(dataframe['项目介绍'].tolist())
DInfo = get_info(details)
Exclamation, QMark = get_mark(details)

detail_short = [detail[:500] for detail in details]
DEmo = sentiment(detail_short)

document_frequency = document_frequency(details)
entropy = [calculate_entropy(detail,details) for detail in details]

dataframe['photo'] = photo
dataframe['DInfo'] = DInfo
dataframe['DEmo'] = DEmo
dataframe['Exclamation'] = Exclamation
dataframe['QMark'] = QMark
dataframe['entropy'] = entropy

# 数据存到新的csv里面
# dataframe.to_csv(readpath + r"processed(ver3.0).csv", index=False)
dataframe.to_csv(r"precessed_2.csv", index=False)

### 标题

In [ ]:
briefs = dataframe['项目简介'].tolist()
BInfo = get_info(briefs)
dataframe['BInfo'] = BInfo
BEmo = sentiment(briefs)

### 项目简介

In [ ]:
titles = process_title(dataframe['项目名称'].tolist())
TInfo = get_info(titles)
TEmo = sentiment(titles)
dataframe['TInfo'] = TInfo
dataframe['TEmo'] = TEmo

### 项目介绍

In [ ]:
photo, details = process_detail(dataframe['项目介绍'].tolist())
DInfo = get_info(details)
Exclamation, QMark = get_mark(details)
document_frequency = document_frequency(details)
entropy = [calculate_entropy(detail,details) for detail in details]
dataframe['entropy'] = entropy

In [ ]:
dataframe.to_csv(readpath + r"processed(ver4.0).csv", index=False)